<a href="https://colab.research.google.com/github/yangxiaoming5201/hello-world/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **挂载硬盘**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 引入外部库



In [4]:
#  !pip install  Queue 

# **库函数**

In [5]:
%tensorflow_version 2.x
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os,datetime
import seaborn as sns
import tensorflow as tf
from queue import Queue
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM,Dropout,Dense,Activation
from IPython.display import HTML
from tensorflow.keras.regularizers import L2
from sklearn.metrics import mean_squared_error
from google.colab import widgets


In [6]:
#指定默认字体
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.family']='sans-serif'
#解决负号'-'显示为方块的问题
mpl.rcParams['axes.unicode_minus'] = False
print('Tensorflow is version:',tf.__version__)
pd.set_option('display.max_rows',30)
pd.set_option('display.max_columns',10)
pd.set_option('colheader_justify', 'center')

Tensorflow is version: 2.4.1


# **调用GPU**

In [7]:
device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# **参数**

In [8]:
#自可变参数
#@title 全局变量参数 { form-width: "350px", display-mode: "both" }
#数据存放路径
root_path = "/content/drive/MyDrive/Colab Notebooks/AirQ" #@param {type:"string"}
site_list=['100'+str(i+1)+'A' for i in range(12)]
train_date = "2019-12-31" #@param {type:"date"}
test_date = "2020-03-31" #@param {type:"date"}
valid_date = "2020-06-30" #@param {type:"date"}
train_site =  48960#@param {type:"number"}
test_site = 51144 #@param {type:"number"}


In [9]:
#不可手动修改参数
file_list=[]#各个站点数据集的路径列表：12个站点
data_list=[]#所有的数据列表;形状：（站点数，时间长度（53328）,输入变量）
std_parmas_list = []  #标准化的相关参数,方便反标准化(均值/标准差)
std_data_list = []  #标准化后的数据列表
train_data_list=[]  #标准化后划分的训练集
test_data_list=[]   #标准化后划分的测试集
valid_data_list=[]  #标准化后划分的验证集


In [10]:
#@title 模型参数 { form-width: "500px", display-mode: "both" }
timesteps=24 #@param {type:"number"}
predict_steps=3 #@param {type:"number"}
epochs =  60#@param {type:"number"}
batchs =32#@param {type:"number"}
init_learn_rate = 0.005 #@param {type:"number"}
droupout_p = 0.5 #@param {type:"number"}
L2_rate =  0.001#@param {type:"number"}
lstm_hide_deep=1
lstm_hide_nums=64

#学习率相关设置
rampup_epochs = 6
sustain_epochs = 0
start_lr = 0.001
min_lr = 0.000001
max_lr = 0.005
exp_decay = 0.85

# **数据处理**

In [11]:

def look_list_shape(look_list):
    """
    查看list的维度
    """
    print(np.array(look_list).shape)

In [12]:
file_list = [os.path.join(root_path, file) for file in os.listdir(root_path)]
file_list.sort()
file_list

['/content/drive/MyDrive/Colab Notebooks/AirQ/1001A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1002A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1003A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1004A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1005A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1006A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1007A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1008A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1009A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1010A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1011A_AirQ.xls',
 '/content/drive/MyDrive/Colab Notebooks/AirQ/1012A_AirQ.xls']

# **读取所有数据**

In [ ]:
# 
def read_data():
  data_list.clear()#
  for file in file_list:
      data =pd.read_excel(file,usecols=[0, 2, 3, 4, 6, 8, 10, 12, 16])
      data.date=data.date.astype("str")
      data.date=pd.to_datetime(data.date,format='%Y-%m-%d')
      # 将CO放大1000背
      data.CO=data.CO*1000
      data_list.append(data)
read_data()

# **查看数据**

In [ ]:
def look_data(datalist):
  tb = widgets.TabBar(site_list)
  i=0
  for data in datalist:
    with tb.output_to(i, select=(i == 0)):
      print(data.head(24))
      i+=1
look_data(data_list)

#**相关性热力分析**

In [ ]:
def data_heatmap(datalist):
  tb = widgets.TabBar(site_list)
  i=0
  for data in datalist:
    new_data=data.set_index(['date','hour'])
    with tb.output_to(i, select=(i == 0)):
      corr=new_data.corr()
      plt.figure( figsize=(10,6))
      sns.heatmap(corr,annot=True)
      plt.show()
      i+=1
data_heatmap(data_list)

# **删除O3数据**

In [ ]:
for data in data_list:
  try:
    data.drop(['O3'],inplace=True,axis=1)
  except:
    pass
print("O3数据已经剔除")

# **查看数据的描述信息**

In [ ]:
def look_data_desc(datalist): 
  tb = widgets.TabBar(site_list)
  i=0
  for data in datalist:
    new_data=data.set_index(['date','hour'])
    with tb.output_to(i, select=(i == 0)):
      print(new_data.describe())
      i+=1
look_data_desc(data_list)

# **原始数据可视化**

In [ ]:
def data_plot(datalist): 
  tb = widgets.TabBar(site_list)
  i=0
  for data in datalist:
    new_data=data.set_index(['date','hour'])
    with tb.output_to(i, select=(i == 0)):
      new_data.plot(subplots=True,figsize=(25,15))
      plt.show()
      i+=1
data_plot(data_list)

# **查看缺失值**

In [ ]:
def look_nan(datalist):
  tb = widgets.TabBar(site_list)
  i=0
  for data in datalist:
    new_data=data.set_index(['date','hour'])
    with tb.output_to(i, select=(i == 0)):
      print(new_data.isnull().sum())
      i+=1
look_nan(data_list)

# **查看0值**

In [ ]:
def look_zero(datalist):
  tb = widgets.TabBar(site_list)
  i=0
  for data in datalist:
    new_data=data.set_index(['date','hour'])
    with tb.output_to(i, select=(i == 0)):
      print(new_data)
      print((new_data==0).astype(int).any())
      i+=1
look_zero(data_list)

# **处理0值**

In [ ]:
def ffill_zero(datalist):
  for data in datalist:
    data.replace(0,np.nan,inplace=True)
    data['hour'].fillna(0,inplace=True)
    data.fillna(method="ffill",inplace=True)
ffill_zero(data_list)
look_zero(data_list)

# **箱型图查看数据平稳性**

In [ ]:
def data_boxplot(datalist):
  tb = widgets.TabBar(site_list)
  i=0
  for data in datalist:
    new_data=data.set_index(['date','hour'])
    with tb.output_to(i, select=(i == 0)):
      plt.figure(figsize=(10,10))
      new_data.CO/=100
      new_data.boxplot()
      plt.show()
      i+=1
data_boxplot(data_list)

# **处理离群点**

In [ ]:
class ThreeQueue(list):
    def __init__(self,len):
        super().__init__()
        self.len=len

    def add(self, num):
        if len(self) == self.len:
            self.pop(0)
        self.append(num)

In [ ]:
outer_distance=3#选取多大步长处理离群点
def data_quantile(datalist):
  data_list_outer=[]
  tb = widgets.TabBar(site_list)
  i=0
  for data in datalist:
    new_data=data.set_index(['date','hour'])
    with tb.output_to(i, select=(i == 0)):
      quantile=new_data.quantile([0.25,0.5,0.75], numeric_only=True)
      one_four=quantile[quantile.index==0.25].reset_index(drop=True)
      one_two=quantile[quantile.index==0.5].reset_index(drop=True)
      third_four=quantile[quantile.index==0.75].reset_index(drop=True)
      outer=one_two+(third_four-one_four)*1.5
      print('各个特征值离群点判定值：\n\n',outer)
      new_data=new_data[new_data<=outer.iloc[0,:]]
      print('各组特征值离群点数如下：\n',pd.isna(new_data).astype(int).sum())

      rows=len(new_data)
      cols=len(new_data.columns)

      for col in range(cols):
        #调试----------
        # col=2
        # ----------------

        tq_front=ThreeQueue(outer_distance)
        tq_after=ThreeQueue(outer_distance)
        j=0#确定后面下一个数位置

        
        for row in range(rows):
          #调试----------
          # if row==1000:
          #     return
          # ----------------

          loc_num=new_data.iloc[row,col]
          if row==0:
            while len(tq_after)<outer_distance:
              num=new_data.iloc[j,col]
              if not pd.isna(num):
                tq_after.add(num)
              j+=1

          if pd.isna(loc_num):
            new_data.iloc[row,col]=np.mean(tq_front+tq_after)

            # -------调试---------
            # print(tq_front,tq_after)
            # print(j)
            # print( new_data.iloc[row,col])
            # ----------------
            
          else:
             while j < rows:
               num=new_data.iloc[j,col]
               if not pd.isna(num):
                 tq_after.add(num)
                 j+=1
                 break
               j+=1
          tq_front.add(new_data.iloc[row,col])
      new_date=new_data.reset_index()
      data_list_outer.append(new_date)
      filename=f'/content/drive/MyDrive/Colab Notebooks/Outer Data/{site_list[i]}_AirQ.csv'
      print(filename+' 已被保存')
      new_date.to_csv(filename,index=False)
      i+=1
  return data_list_outer
data_list[0]
data_list_outer=data_quantile(data_list)#返回处理过离群点后的数据

 相关检验

In [ ]:
data_boxplot(data_list_outer)

# **对数化处理**

In [ ]:
def data_log10(datalist):
  data_log10_list=[]
  tb = widgets.TabBar(site_list)
  i=0
  for data in datalist:
    new_data=data.set_index(['date','hour'])
    with tb.output_to(i, select=(i == 0)):
      new_data = new_data.replace('[0-1]', 1, regex=True)
      new_data_log10=np.log10(new_data).reset_index()
      data_log10_list.append(new_data_log10)
      print(new_data_log10)
      i+=1
  return data_log10_list
data_log10_list=data_log10(data_list_outer)#返回经过对数化处理的列表数据

# **反对数化**

In [ ]:
def anti_log10_data(datalist):
  anti_log10_list=[]
  for data in datalist:
    new_data=np.power(10,data)
    anti_log10_list.append(new_data)
  return anti_log10_list


相关检验

In [ ]:
data_heatmap(data_log10_list)

# **数据缩放(归一化：0，标准化：1)**

In [ ]:
zoom_method=1
def data_zoom(datalist,method=0):
  tb = widgets.TabBar(site_list)
  i=0
  zoom_class_list=[]
  zoom_data_list=[]
  for data in datalist:
    with tb.output_to(i, select=(i == 0)):
      new_data=data.set_index(['date','hour'])
      print(new_data)
      if method==0:
        style=MinMaxScaler()
      else:
        style=StandardScaler()
      style.fit(new_data)
      zoom_class_list.append(style)
      nl_data=style.transform(new_data)
      print(nl_data)
      zoom_data_list.append(nl_data)
      i+=1
  return zoom_data_list,zoom_class_list
zoom_data_list,zoom_class_list=data_zoom(data_log10_list,zoom_method)


# **反数据缩放**

In [ ]:
def data_anti_zoom(datalist,class_list):
  tb = widgets.TabBar(site_list)
  i=0
  anti_zoom_data=[]
  for data,style in zip(datalist,class_list):
    with tb.output_to(i, select=(i == 0)):
      anti_data=style.inverse_transform(data)
      anti_zoom_data.append(anti_data)
      # print(anti_data)
      i+=1
  return anti_zoom_data
anti_zoom_data=data_anti_zoom(zoom_data_list,zoom_class_list)


In [ ]:
data1=data_list[1]
print(data1[data1.date==train_date],'\n---------------------------\n')
print(data1[data1.date==test_date],'\n---------------------------\n')

# **划分数据集**(训练集、验证集、测试集)

In [ ]:
def split_dataset(dataset_list):
    """
    划分数据集
    """
    train_data_list.clear()  #标准化后划分的训练集
    test_data_list.clear()   #标准化后划分的测试集
    valid_data_list.clear()  #标准化后划分的验证集 
    
    for dataset in dataset_list:
        train_data=dataset[:train_site]
        train_data_list.append(train_data)

        test_data=dataset[train_site:test_site]
        test_data_list.append(test_data)

        valid_data=dataset[test_site:]
        valid_data_list.append(valid_data)
        # print(trainX.shape,testX.shape,validX.shape,'-----------\n'
split_dataset(zoom_data_list)


In [ ]:
look_list_shape(train_data_list)
look_list_shape(test_data_list)
look_list_shape(valid_data_list)
(44544/24,6600/24,2184/24)
test_data_list[0]

# **划分特征集与目标集**

In [ ]:
def get_X_Y(data):
  x_y_list=[]
  for dataset in data:
    x_y_tuple=()
    datax=[]#构造x
    datay=[]#构造y
    for each in range(0,len(dataset)-timesteps - predict_steps,predict_steps):
        x = dataset[each:each+timesteps]
        y = dataset[each+timesteps:each+timesteps+predict_steps]
        datax.append(x)
        datay.append(y)
    x_y_tuple=(np.array(datax),np.array(datay))
    x_y_list.append(x_y_tuple)
  return x_y_list





  

In [ ]:
train_x_y_list=get_X_Y(train_data_list)

test_x_y_list=get_X_Y(test_data_list)

valid_x_y_list=get_X_Y(valid_data_list)

In [ ]:
for train_x_y_tuple,valid_x_y_tuple,test_x_y_tuple in zip(train_x_y_list,valid_x_y_list,test_x_y_list):
  train_x=train_x_y_tuple[0]
  train_y=train_x_y_tuple[1]
  valid_x=valid_x_y_tuple[0]
  valid_y=valid_x_y_tuple[1]
  test_x=test_x_y_tuple[0]
  test_y=test_x_y_tuple[1]
  print(train_x.shape,train_y.shape,test_y.shape)



In [ ]:
input_dim=train_x_y_list[0][0].shape[2]
input_dim#输入维度

# **构建学习率下降函数**

In [ ]:
# def scheduler(epoch):
#     # 前5个epoch学习率保持不变，5个epoch后学习率按比例衰减
#     if epoch < 5:
#         return init_learn_rate
#     else:
#         lr = init_learn_rate * tf.math.exp(0.1 * (5 - epoch))
#         return lr.numpy()

def scheduler(epoch):
  if epoch < rampup_epochs:
    return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
  elif epoch < rampup_epochs + sustain_epochs:
    return max_lr
  else:
    return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
    
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda epoch: scheduler(epoch), verbose=True)
rang = np.arange(epochs)
y = [scheduler(x) for x in rang]
plt.plot(rang, y)
print('Learning rate per epoch:')

# **构建DM-LSTM模型**

In [ ]:
# 使用3层LSTM，输出层为2输出的Dense层

def create_model():
  model = Sequential()
  model.add(LSTM(64 ,activation='tanh',recurrent_activation='sigmoid',bias_initializer='zero',unit_forget_bias=True,input_shape=(timesteps,input_dim),use_bias=True,recurrent_regularizer=L2(L2_rate),return_sequences=True))
  model.add(Dropout(droupout_p))
  for i in range(lstm_hide_deep):
    if i==lstm_hide_deep-1:
      model.add(LSTM(lstm_hide_nums,activation='tanh',recurrent_activation='sigmoid',bias_initializer='zero',unit_forget_bias=True,recurrent_regularizer=L2(L2_rate),use_bias=True, return_sequences=False))
      model.add(Dropout(droupout_p))
      break
    model.add(LSTM(lstm_hide_nums ,activation='tanh',recurrent_activation='sigmoid',bias_initializer='zero',unit_forget_bias=True,input_shape=(timesteps,input_dim),use_bias=True,recurrent_regularizer=L2(L2_rate),return_sequences=True))
    model.add(Dropout(droupout_p))
  # model.add(Activation('sigmoid'))1
  model.add(Dense(predict_steps*input_dim))
  model.compile(loss='mean_squared_error', optimizer='adam')
  return model

model=create_model()
model.summary()

# **打印模型结构图**

In [ ]:
tf.keras.utils.plot_model(model,show_shapes = True,show_dtype=True,expand_nested=True)

# **训练模型**

In [ ]:
def train_model():
  # if os.path.exists("./drive/MyDrive/checkpoints/checkpoint"):
  #     print("《《《《《《《《《《《《《训练的模型权重已经被加载》》》》》》》》》》》")
  #      model.load_weights(f'./drive/MyDrive/checkpoints/MyModel_t+{predict_steps}')
  tb = widgets.TabBar(site_list)
  i=0
  for train_x_y_tuple,valid_x_y_tuple in zip(train_x_y_list,valid_x_y_list):
     with tb.output_to(i, select=(i == 0)):
        train_x=train_x_y_tuple[0]
        train_y=train_x_y_tuple[1]
        valid_x=valid_x_y_tuple[0]
        valid_y=valid_x_y_tuple[1]
        # logdir=os.path.join("logs",datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
        # tc=tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1)
        history =model.fit(train_x, train_y.reshape(train_y.shape[0],-1),
                          epochs=epochs,
                          batch_size=batchs,
                          verbose=1,
                          validation_data=(valid_x, valid_y.reshape(valid_y.shape[0],-1)),callbacks=[reduce_lr])
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        print(loss)
        plt.plot(range(len(loss)), loss, 'b-', label='loss')
        plt.plot(range(len(loss)), val_loss, 'r-', label='val_loss')
        plt.legend(loc='best')
        plt.show()
        print(site_list[i]+'步Model已经保存')
        model.save_weights(f'./drive/MyDrive/checkpoints/MyModel_t+{predict_steps}')
        i+=1
train_model()

# **获取预测的真实比例值**

In [ ]:
def get_predict_data(datalist):
  global zoom_method,model
  i=0
  y_pre_zoom_list=[]
  y_true_zoom_list=[]
  #直接的预测数据
  for data in datalist:
      x_true=data[0]
      y_true=data[1]
      y_true=y_true.reshape(y_true.shape[0],-1)
      y_true_zoom_list.append(y_true.reshape(-1,input_dim))

      y_pre=model.predict(x_true)
      y_pre=y_pre.reshape(-1,input_dim)
      # print(y_pre.shape)
      y_pre_zoom_list.append(y_pre)
      # lo=np.sqrt(mean_squared_error(testy,pre_y))
      
  #反缩放的预测数据
  
  y_pre_log10_list=data_anti_zoom(y_pre_zoom_list,zoom_class_list)
  y_true_log10__list=data_anti_zoom(y_true_zoom_list,zoom_class_list)

  #反对数化数据
  y_pre_list=anti_log10_data(y_pre_log10_list)
  y_true_list=anti_log10_data(y_true_log10__list)
  return y_pre_list,y_true_list
y_pre_list,y_true_list=get_predict_data(test_x_y_list)
print(y_true_list[0].shape,y_pre_list[0].shape)

# **做出对比图**

In [ ]:
def pre_true_plot(true_list,pre_list):
  tb = widgets.TabBar(site_list)
  i=0
  for true_data,pre_data in zip(true_list,pre_list):
     with tb.output_to(i, select=(i == 0)):
       
       print('mse:',mean_squared_error(true_data,pre_data))
       print('rmse:',np.sqrt(mean_squared_error(true_data,pre_data)))
       
       plt.figure(figsize=(15,10))
       plt.plot(true_data[:500,1],label='true data')
       plt.plot(pre_data[:500,1],label='predict data')
       plt.legend()
       plt.show()
       i+=1
pre_true_plot(y_true_list,y_pre_list)


In [ ]:
model.load_weights(f'./drive/MyDrive/checkpoints/MyModel_t+{predict_steps}')

# **生成真实值与预测值的折线图**